In [91]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from splinter.exceptions import ElementDoesNotExist
import re
import requests
import time
from decimal import Decimal
from re import sub
import geojson
import numpy as np

In [67]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Scrape Craigslist Freestuff

In [8]:
# Go to sacramento cl and navigate to 1st item

url = "https://sacramento.craigslist.org/d/free-stuff/search/zip"
browser.visit(url)
browser.click_link_by_partial_href('https://sacramento.craigslist.org')
time.sleep(2)

In [11]:
# Using Splinter to scrape cl and store data in dataframe stuff (clean up condition)

stuff = pd.DataFrame(columns=['lat', 'long','age', 'title'])

for x in range(1, 4):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find(id="titletextonly").text
    age = soup.find('time', class_="date timeago")["datetime"]
    loc = soup.find(id='map')
    lat = loc["data-latitude"]
    long = loc["data-longitude"]
    
    stuff.loc[x] = [lat, long, age, title]
    time.sleep(2)
    browser.click_link_by_partial_text('next')

print(stuff)

         lat         long                       age                      title
1  38.807876  -121.354025  2019-10-27T17:34:04-0700       Free GE Refrigerator
2  38.686287  -121.190014  2019-10-27T17:30:33-0700  Free flower window basket
3  38.601292  -121.406004  2019-10-27T17:27:20-0700                   TV stand


In [82]:
# format for output to geojson

stuff["lat"] = pd.to_numeric(stuff["lat"])
stuff["long"] = pd.to_numeric(stuff["long"])
stuff["evel"] = 1
stuff = stuff.reindex(columns=['lat', 'long', 'elev', 'title', 'age'])

stuff.head()

,lat,long,elev,title,age
1,38.807876,-121.354025,0,Free GE Refrigerator,2019-10-27T17:34:04-0700
2,38.686287,-121.190014,0,Free flower window basket,2019-10-27T17:30:33-0700
3,38.601292,-121.406004,0,TV stand,2019-10-27T17:27:20-0700


In [8]:
# (pd.DataFrame.from_dict(data=stuff2, orient='index')
  #   .to_csv('stuff.csv', header=False))

In [60]:

import scrape_cl

things = scrape_cl.scrape()
print(things)

         lat         long                       age  \
1  38.269100  -121.300000  2019-10-07T08:04:28-0700   
2  38.787700  -121.236600  2019-10-27T19:17:57-0700   
3  38.553900  -121.369300  2019-10-27T19:03:29-0700   

                         title  
1                   FREE BOATS  
2            Fluorescent bulbs  
3  Here tonight gone tomorrow!  


In [61]:
# Go to melissa.com and find zip codes

look_up_url = "https://www.melissa.com/v2/lookups/latlngzip4/"
browser.visit(look_up_url)


In [64]:
zips = []

for index, row in things.iterrows():
    browser.fill('lat', row['lat'])
    browser.fill('lng', row['long'])
    browser.find_by_value('Submit').first.click()
    element = browser.find_by_css("tbody")
    zip_code = element.value.split("Postal Code ")[1].split("-")[0]
    zips.append(zip_code)
    
se = pd.Series(zips)
things['zip'] = se.values
     
things.head()

,lat,long,age,title,zip
1,38.269100,-121.300000,2019-10-07T08:04:28-0700,FREE BOATS,95632
2,38.787700,-121.236600,2019-10-27T19:17:57-0700,Fluorescent bulbs,95677
3,38.553900,-121.369300,2019-10-27T19:03:29-0700,Here tonight gone tomorrow!,95826


In [74]:
inc_data_url = "https://zipatlas.com/us/ca/sacramento/zip-code-comparison/median-household-income.htm"

browser.visit(inc_data_url)

inc_data = browser.find_by_css("tbody")

# inc_data.value

In [78]:
data_list = inc_data.value.split("National Rank")[1].split("28 total")[0]
zips = []
incomes = []
data_list = data_list.split(" ")
for i in range(1, 28):
    zips.append(data_list[8*(i-1)+1])
    incomes.append(Decimal(sub(r'[^\d.]', '', data_list[8*(i-1)+7])))

zip_incomes = {"zip":zips, "income":incomes}
zip_inc = pd.DataFrame(zip_incomes)

# zip_inc.head()

In [92]:
data = things.merge(zip_inc,how='left', on='zip')
data['elev'] = 0
data = data.reindex(columns=['lat', 'long', 'elev', 'title', 'age', 'zip', 'incomes'])
data["lat"] = pd.to_numeric(data["lat"])
data["long"] = pd.to_numeric(data["long"])
data = data.replace(np.nan, '', regex=True)
data.head()

,lat,long,elev,title,age,zip,incomes
0,38.2691,-121.3000,0,FREE BOATS,2019-10-07T08:04:28-0700,95632,
1,38.7877,-121.2366,0,Fluorescent bulbs,2019-10-27T19:17:57-0700,95677,
2,38.5539,-121.3693,0,Here tonight gone tomorrow!,2019-10-27T19:03:29-0700,95826,42562.00


In [89]:
#define method to create geojson file from dataframe

def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"], 
                                                    X["elev"])),
                            properties=dict(age=X["age"],
                                            title=X["title"],
                                            zip_code=X["zip"],
                                            income=X["incomes"])))
    df.apply(insert_features, axis=1)
    with open('static/data/stuff.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)


In [93]:
# data2geojson(stuff)

data2geojson(data)